In [1]:
import numpy as np
from hmmlearn import hmm


In [2]:
hmm_states = 2

model = hmm.CategoricalHMM(n_components=hmm_states)
model.emissionprob_ = np.array([[0.9,0.1],[0.1,0.9]])
model.transmat_ = np.array([[0.9,0.1],[0.1,0.9]])
model.startprob_ = np.array([0.4,0.6])
data,states = model.sample(n_samples = 1000,random_state=28)
data = np.squeeze(data)

In [3]:
# model2 = hmm.CategoricalHMM(n_components=hmm_states,n_iter=1000,implementation='log')
# model2.fit(data.reshape(-1,1))
# print(model2.emissionprob_,model2.startprob_,model2.transmat_)
# # print("Most likely hidden states:", hidden_states)


In [4]:
# model2.monitor_.converged

In [5]:
n_states = 2
n_features = 2
n_obs = data.shape[0]
# emission = np.random.rand(n_obs,n_states)
# emission = emission/np.tile(np.expand_dims(np.sum(emission,axis=1),axis=1),(1,2))

emission = np.random.rand(n_states,n_features)
emission = emission/np.tile(np.expand_dims(np.sum(emission,axis=1),axis=1),(1,2))
emission = np.array([[0.96515917, 0.03484083],
       [0.36020674, 0.63979326]])


transition =  np.random.rand(n_states,n_states)
transition = transition/np.tile(np.expand_dims(np.sum(transition,axis=1),axis=1),(1,2))
transition = np.array([[0.48577716, 0.51422284],
       [0.41604097, 0.58395903]])


scale_factors = np.zeros((n_obs))
forward_hat = np.zeros((n_obs,n_states))

backward_hat = np.zeros((n_obs,n_states))

init_prob = np.array([0.5,0.5])


p_old = -10000
tol = 0.01
max_iter = 1000

In [6]:
print(f'emission is {emission} and transition is {transition}')

emission is [[0.96515917 0.03484083]
 [0.36020674 0.63979326]] and transition is [[0.48577716 0.51422284]
 [0.41604097 0.58395903]]


In [ ]:
for ite in range(max_iter):
    print(ite)
    forward_hat = np.zeros((n_obs,n_states))
    backward_hat = np.zeros((n_obs,n_states))
    scale_factors = np.zeros((n_obs))


    forward_hat[0,:] = init_prob * emission[:,data[0]]
    scale_factors[0] = np.sum(forward_hat[0,:])
    forward_hat[0,:] = forward_hat[0,:]/scale_factors[0]
    
    for t in range(n_obs-1):
        temp = np.matmul(forward_hat[t,:] ,transition) * emission[:,data[t+1]]
        scale_factors[t+1] = np.sum(temp)
        forward_hat[t+1,:] = temp/scale_factors[t+1]
        # print(f'temp is {temp} and the scale factor is {scale_factors[t+1]} and the forward_hat is {forward_hat[t+1]}')


    backward_hat[-1,:] = scale_factors[-1]
    for t in reversed(range(n_obs-1)):
        temp = np.matmul(backward_hat[t+1,:]*emission[:,data[t+1]],transition.T)
        backward_hat[t,:] = temp/scale_factors[t]
        


    a = np.zeros((n_obs,n_states))
    b = np.zeros((n_obs,n_states,n_states))
    for i in range(n_obs):
        for j in range(n_states):
            a[i,j] = forward_hat[i,j]*backward_hat[i,j]
        temp = np.sum(a[i,:])
        a[i,:] = a[i,:]/temp

    for t in range(n_obs-1):
        for i in range(n_states):
            for j in range(n_states):
                b[t,i,j] = scale_factors[t+1]*forward_hat[t,i]*backward_hat[t+1,j]*transition[i,j]*emission[j,data[t+1]]

    for i in range(n_states):
        for j in range(n_states):
            transition[i,j] = np.sum(b[0:-1,i,j])/np.sum(b[0:-1,i,:])
            # print(np.sum(b[0:-1,i,j]),np.sum(b[0:-1,i,:]))

    for i in range(n_states):
        init_prob[i] = a[0,i]/np.sum(a[0,:])

    for i in range(n_states):
        for j in range(n_states):
            emission[j,i] = np.sum(a[np.argwhere(data==i),j]) / np.sum(a[:,j])
            
    p = np.sum(np.log(scale_factors))
    print(f'p is:{p},transition is {transition},emission is {emission},init is {init_prob}')
    # print(f'transition prob is: {transition}')
    # print(f'emission prob is:{emission}')
    if p>p_old and p - p_old < tol:
        break
    p_old = p

0
p is:-715.9362946448327,transition is [[0.4485059  0.5514941 ]
 [0.19858759 0.80141241]],emission is [[0.94779663 0.05220337]
 [0.23891298 0.76108702]],init is [0.04623385 0.95376615]
1
p is:-651.1581967678944,transition is [[0.64676852 0.35323148]
 [0.23674521 0.76325479]],emission is [[0.96950968 0.03049032]
 [0.23883302 0.76116698]],init is [0.00234453 0.99765547]
2
p is:-608.7530984061144,transition is [[0.6445439  0.3554561 ]
 [0.19690183 0.80309817]],emission is [[0.97562647 0.02437353]
 [0.17688965 0.82311035]],init is [4.48201533e-05 9.99955180e-01]
3
p is:-593.1411598031319,transition is [[0.67709821 0.32290179]
 [0.24043024 0.75956976]],emission is [[0.98166872 0.01833128]
 [0.14982839 0.85017161]],init is [5.99734645e-07 9.99999400e-01]
4
p is:-588.8323521608793,transition is [[0.63647095 0.36352905]
 [0.25582719 0.74417281]],emission is [[0.98438619 0.01561381]
 [0.1205566  0.8794434 ]],init is [5.58821444e-09 9.99999994e-01]
5
p is:-594.4578116945606,transition is [[0.64